# API for GENSCAN

[GENSCAN](http://hollywood.mit.edu/GENSCAN.html) identifies complete exon/intron structures of genes in genomic DNA. Novel features of the program include the capacity to predict multiple genes in a sequence, to deal with partial as well as complete genes, and to predict consistent sets of genes occurring on either or both DNA strands. GENSCAN is shown to have substantially higher accuracy than existing methods when tested on standardized sets of human and vertebrate genes, with 75 to 80% of exons identified exactly. The program is also capable of indicating fairly accurately the reliability of each predicted exon

**Task:** Here we will write API for GENSCAN Online Version.

**Input data:** 
* `neurofibromin.fa` contains neurofibromin 1 nucleotide sequence.<br>
NF1 spans over 350-kb of genomic DNA and contains 62 exons.
* `insulin.fa` contains insulin gene nulceotide sequence.<br>
The human insulin gene (INS) is a small gene located on chromosome 11 and is composed of 3 exons separated by two introns.
* `nlrp3.fa` encodes a pyrin-like protein containing a pyrin domain, a nucleotide-binding site (NBS) domain, and a leucine-rich repeat (LRR) motif.<br>
NLRP3 maps on chromosome 1 and consists of 3108 nucleotides.

## Libraries

In [1]:
import re

import requests
from IPython.display import HTML

## Functions

In [2]:
class GenscanOutput:

    def __init__(self, status, cds_list, intron_list, exon_list, resp):
        '''
        :param status: status of response (200 - if it was successful)
        :param cds_list: list with predicted protein sequences after splicing
        :param intron_list: list of introns coordinates
        :param exon_list: list of exons coordinates
        :param resp: response from the site (after implementation of requests.get() function)
        '''
        
        self.status = status
        self.intron_list = intron_list
        self.exon_list = exon_list
        self.cds_list = cds_list
        self.resp = resp


def run_genscan(sequence=None,
                sequence_file=None,
                organism="Vertebrate",
                exon_cutoff=1.00,
                sequence_name="",
                print_options='Predicted peptides only'):
    '''
    API for GENSCAN Online Version.
    It identifies complete exon/intron structures of genes in genomic DNA.
    Allows to analyze sequences up to 1 million base pairs (1 Mbp) in length.

    :param sequence: str, DNA sequence (upper or lower case, spaces/numbers ignored)
    :param sequence_file: str, path to DNA sequence file
    :param organism: str, type of organism from which DNA sequence was obtained
    :param exon_cutoff: float, allows to find less probable exons.
                        Available values: 1.00, 0.5, 0.25, 0.1, 0.05, 0.02, 0.01 (default 1.00)
    :param sequence_name: name of the target nucleotide sequence
    :param print_options: str, 'Predicted peptides only' (default) or 'Predicted CDS and peptides'
    '''

    genscan_url = 'http://hollywood.mit.edu/cgi-bin/genscanw_py.cgi'
    payload = {'-o': organism,
               '-e': exon_cutoff,
               '-n': sequence_name,
               '-p': print_options}

    # make request and get info from browser
    if sequence_file is None:
        payload['-s'] = sequence
        resp = requests.post(genscan_url, data=payload)
    else:
        with open(sequence_file, 'rb') as fasta_file:
            files = {'-u': fasta_file}
            resp = requests.post(genscan_url, data=payload, files=files)

    # find list of coding sequences
    pattern_1 = re.compile(r'Predicted peptide sequence\(s\):[\d\D]+')
    pattern_2 = re.compile(r'>.+\n{2}([ARNDCEQGHILKMFPSTWYV\n]+)')
    pattern_3 = re.compile(r'([ARNDCEQGHILKMFPSTWYV]+)')
    results = pattern_2.findall(pattern_1.findall(resp.text)[0])
    cds_list = []
    for result in results:
        cds_list.append(''.join(pattern_3.findall(result)))

    # find exons and introns
    exon_list = []
    intron_list = []
    for element in re.compile(r'\d\.\d{2}\s+([A-z]+)\s+[\-\+]\s+(\d+)\s+(\d+)').findall(resp.text):
        if element[0] in ['Init', 'Intr', 'Term', 'Sngl']:
            exon_list.append(element[1:])
        else:
            intron_list.append(element[1:])

    genscan_output = GenscanOutput(status=resp.status_code,
                                   cds_list=cds_list,
                                   intron_list=intron_list,
                                   exon_list=exon_list,
                                   resp=resp)

    return genscan_output

**Look at GENSCAN Online Version:**

In [3]:
resp = requests.get('http://hollywood.mit.edu/GENSCAN.html')
HTML(data=resp.text)

## Testing API

### Testing on `neurofibromin.fa`

In [4]:
genscan_output_neurofibromin = run_genscan(sequence=None, 
                                           sequence_file='../data/neurofibromin.fa', 
                                           organism="Vertebrate", 
                                           exon_cutoff=1.00, 
                                           sequence_name="neurofibramin 1",
                                           print_options='Predicted peptides only')

**That's like looks browser version for our response:**

In [5]:
HTML(data=genscan_output_neurofibromin.resp.text)

### First table from results (which we parse):
* **Gn.Ex :** gene number, exon number (for reference)
* **Type  :**
        Init = Initial exon
        Intr = Internal exon
        Term = Terminal exon
        Sngl = Single-exon gene
        Prom = Promoter
        PlyA = poly-A signal
* **S     :** DNA strand (+ = input strand; - = opposite strand)
* **Begin :** beginning of exon or signal (numbered on input strand)
* **End   :** end point of exon or signal (numbered on input strand)
* **Len   :** length of exon or signal (bp)
* **Fr    :** reading frame (a codon ending at x is in frame f = x mod 3)
* **Ph    :** net phase of exon (length mod 3)
* **I/Ac  :** initiation signal or acceptor splice site score (x 10)
* **Do/T  :** donor splice site or termination signal score (x 10)
* **CodRg :** coding region score (x 10)
* **P     :** probability of exon (sum over all parses containing exon)
* **Tscr  :** exon score (depends on length, I/Ac, Do/T and CodRg scores)

**Status code:**

In [6]:
genscan_output_neurofibromin.status

200

**Coding sequences:**

In [7]:
print(*genscan_output_neurofibromin.cds_list, sep='\n\n')

MAAHRPVEWVQAVVSRFDEQGLSAGGDGTGGLCTCQCSLDNGHGGIGRVYSRWLQWHGRVLARMCAGGEGKTRYTCAHVWAKHLEGGHGRVPAGKAAWSSLRWEEGIGRLVLFHRDHCGWSTLPVSTLSVQELLPIKTGQQNTHTKVSTEHNKECLINISKYKFSLVISGLTTILKNVNNMQPKDTMRLDETMLVKQLLPEICHFLHTCREGNQHAAELRNSASGVLFSLSCNNFNAVFSRISTRLQELTVCSEDNVDVHDIELLQYINVDCAKLKRLLKETAFKFKALKKVAQLAVINSLEKAFWNWVENYPDEFTKLYQIPQTDMAECAEKLFDLVDGFAESTKRKAAVWPLQIILLILCPEIIQDISKDVVDENNMNKKQILSLKSAKIDLSETLISCRKRAEIVKKQTQTLIMQVADLQQKVHVQPNQVSTVKVWALIGKEWDPATWKGHTYNQTKVLAGVLLWPIYPVTQRTASFEWGPEQEKALQQVQAAVQAALPLGPYDPTDPMVLEVSVADKDAVSSLWQAPIAVKWVVHSSIPSSSGSGTYTIGLKQVLKAQVNGKLQQPHPGRTTNDPDPSGMKVWVTLPGKKPPPAEVLAEGKGNTEWLVEEASVFKFFSFGTWTGSPCSSPCRQPIVGPRDRKLFLDSLRKALAGHGGSRQLTESAAIACVKLCKASTYINWEDNSVIFLLVQSMVVDLKNLLFNPSKPFSRGSQPADVDLMIDCLVSCFRISPHNNQHFKICLAQNSPSTFHYVLVNSLHRIITNSALDWWPKIDAVYCHSVELRNMFGETLHKAVQGCGAHPAIRMAPSLTFKEKVTSLKFKEKPTDLETRSYKYLLLSMVKLIHADPKLLLCNPRKQGPETQGSTAELITGLVQLVPQSHMPEIAQEAMEQADRSSCHFLLFYGVGCDIPSSGNTSQMSMDHEELLRTPGASLRKGKGNSSMDSAAGCSGTPPICRQAQTKLEVALYMFLWNPDTEAVLVAMSCFRHLCEEADI

**Intron coordinates:**

In [8]:
print(*genscan_output_neurofibromin.intron_list, sep='\n\n')

('178153', '178158')

('178310', '178305')

('201623', '201584')

('201698', '201693')

('225597', '225558')

('226591', '226630')

('252323', '252328')

('252544', '252583')

('280755', '280760')


**Exon coordinates:**

In [9]:
print(*genscan_output_neurofibromin.exon_list, sep='\n\n')

('427', '486')

('35748', '36086')

('61100', '61243')

('68303', '68493')

('75008', '75114')

('86539', '86606')

('86827', '86902')

('87625', '87782')

('89844', '90062')

('90329', '90356')

('90791', '90997')

('91195', '91283')

('93179', '93335')

('93962', '94053')

('105539', '105712')

('106154', '106276')

('106528', '106602')

('111357', '111488')

('119568', '119702')

('124122', '124235')

('130212', '130367')

('131552', '131801')

('132335', '132408')

('132640', '132723')

('134142', '134582')

('134952', '135091')

('135377', '135499')

('135959', '136042')

('137190', '137306')

('137817', '137998')

('138119', '138330')

('140728', '140889')

('141035', '141138')

('149707', '149823')

('154101', '154236')

('163461', '163619')

('164149', '164246')

('165486', '165632')

('166828', '166974')

('170346', '170456')

('177762', '177861')

('201139', '200126')

('210597', '209380')

('219161', '219096')

('219582', '219488')

('224130', '223425')

('229474', '229484')

### Testing on `insulin.fa`

In [10]:
genscan_output_insulin = run_genscan(sequence=None, 
                                     sequence_file='../data/insulin.fa', 
                                     organism="Vertebrate", 
                                     exon_cutoff=1.00, 
                                     sequence_name="insulin",
                                     print_options='Predicted peptides only')

**That's like looks browser version for our response:**

In [11]:
HTML(data=genscan_output_insulin.resp.text)

**Status code:**

In [12]:
genscan_output_insulin.status

200

**Coding sequences:**

In [13]:
print(*genscan_output_insulin.cds_list, sep='\n\n')

MALWMRLLPLLALLALWGPDPAAAFVNQHLCGSHLVEALYLVCGERGFFYTPKTRREAEDLQVGQVELGGGPGAGSLQPLALEGSLQKRGIVEQCCTSICSLYQLENYCN


**Intron coordinates:**

In [14]:
print(*genscan_output_insulin.intron_list, sep='\n\n')

('1456', '1461')


**Exon coordinates:**

In [15]:
print(*genscan_output_insulin.exon_list, sep='\n\n')

('283', '469')

('1257', '1402')


### Testing on `nlrp3.fa`

In [16]:
genscan_output_nlrp3 = run_genscan(sequence=None, 
                                   sequence_file='../data/nlrp3.fa', 
                                   organism="Vertebrate", 
                                   exon_cutoff=1.00, 
                                   sequence_name="NLRP3",
                                   print_options='Predicted peptides only')

**That's like looks browser version for our response:**

In [17]:
HTML(data=genscan_output_nlrp3.resp.text)

**Status code:**

In [18]:
genscan_output_nlrp3.status

200

**Coding sequences:**

In [19]:
print(*genscan_output_nlrp3.cds_list, sep='\n\n')

MASTRCKLARYLEDLEDVDLKKFKMHLEDYPPQKGCIPLPRGQTEKADHVDLATLMIDFNGEEKAWAMAVWIFAAINRRDLYEKAKRDEPKWGSDNARVSNPTVICQEDSIEEEWMGLLEYLSRISICKMKKDYRKKYRKYVRSRFQCIEDRNARLGESVSLNKRYTRLRLIKEHRSQQEREQELLAIGKTKTCESPVSPIKMELLFDPDDEHSEPVHTVVFQGAAGIGKTILARKMMLDWASGTLYQDRFDYLFYIHCREVSLVTQRSLGDLIMSCCPDPNPPIHKIVRKPSRILFLMDGFDELQGAFDEHIGPLCTDWQKAERGDILLSSLIRKKLLPEASLLITTRPVALEKLQHLLDHPRHVEILGFSEAKRKEYFFKYFSDEAQARAAFSLIQENEVLFTMCFIPLVCWIVCTGLKQQMESGKSLAQTSKTTTAVYVFFLSSLLQPRGGSQEHGLCAHLWGLCSLAADGIWNQKILFEESDLRNHGLQKADVSAFLRMNLFQKEVDCEKFYSFIHMTFQEFFAAMYYLLEEEKEGRTNVPGSRLKLPSRDVTVLLENYGKFEKGYLIFVVRFLFGLVNQERTSYLEKKLSCKISQQIRLELLKWIEVKAKAKKLQIQPSQLELFYCLYEMQEEDFVQRAMDYFPKIEINLSTRMDHMVSSFCIENCHRVESLSLGFLHNMPKEEEEEEKEGRHLDMVQCVLPSSSHAACSHGLVNSHLTSSFCRGLFSVLSTSQSLTELDLSDNSLGDPGMRVLCETLQHPGCNIRRLWLGRCGLSHECCFDISLVLSSNQKLVELDLSDNALGDFGIRLLCVGLKHLLCNLKKLWLVSCCLTSACCQDLASVLSTSHSLTRLYVGENALGDSGVAILCEKAKNPQCNLQKLGLVNSGLTSVCCSALSSVLSTNQNLTHLYLRGNTLGDKGIKLLCEGLLHPDCKLQVLELDNCNLTSHCCWDLSTLLTSSQSLRKLSLGNNDLGDLGVMMFCEVLKQQSCLLQN

**Intron coordinates:**

In [20]:
print(*genscan_output_nlrp3.intron_list, sep='\n\n')

('32677', '32682')


**Exon coordinates:**

In [21]:
print(*genscan_output_nlrp3.exon_list, sep='\n\n')

('2682', '2958')

('7111', '7230')

('7728', '9480')

('13466', '13636')

('17984', '18154')

('19851', '20021')

('27853', '28023')

('28532', '28702')

('32286', '32385')
